In [ ]:

import os
import sys
import librosa
import soundfile as sf
import numpy as np
import random
import pandas as pd
sys.path.insert(0, os.path.abspath('../package'))
import config_new as config
import IPython.display as ipd

def normalize(audio, target_level=-20.0):

    rms = np.sqrt(np.mean(audio**2))  # Compute RMS
    if rms > 0:  # Avoid division by zero
        scalar = 10 ** (target_level / 20) / rms
        audio *= scalar
    return np.clip(audio, -1.0, 1.0)



# def get_mosquito_species(mosquito_dirs):

#     all_files = []
    
#     for mosquito_dir in mosquito_dirs:
#         if not os.path.exists(mosquito_dir):
#             print(f"❌ Path does not exist: {mosquito_dir}")
#             continue  
        
#         files = [os.path.join(mosquito_dir, f) for f in os.listdir(mosquito_dir) if f.endswith(".wav") and not f.startswith("An.Minimus")]
#         all_files.extend(files)  

#     if not all_files:
#         raise ValueError("⚠️ No mosquito files found!")

#     selected_file = random.choice(all_files)  # ✅ สุ่มไฟล์ 1 ไฟล์
#     species = os.path.basename(selected_file).split("_")[0]  # ✅ ดึง species ของไฟล์
#     print(f"🎯 Selected File: {selected_file} -> Species: {species}")

#     return selected_file, species  # ✅ คืนค่าไฟล์ + species ที่ได้
def get_mosquito_species(mosquito_dirs):
    all_species_files = {}

    for mosquito_dir in mosquito_dirs:
        if not os.path.exists(mosquito_dir):
            continue  

        for f in os.listdir(mosquito_dir):
            if f.endswith(".wav") and not f.startswith("An.Minimus"):
                species = f.split("_")[0]  
                if species not in all_species_files:
                    all_species_files[species] = []
                all_species_files[species].append(os.path.join(mosquito_dir, f))

    if not all_species_files:
        raise ValueError("⚠️ No mosquito files found!")

    # ✅ ใช้ weighted sampling เพื่อให้ทุก species มีโอกาสถูกเลือกเท่ากัน
    species = random.choice(list(all_species_files.keys()))
    selected_file = random.choice(all_species_files[species])

    # print(f"🎯 Selected File: {selected_file} -> Species: {species}")
    return selected_file, species



def get_environment_type(species):

    # print(f"🐝 Checking environment for species: {species}")

    for env, settings in config.ENVIRONMENTS.items():
        if species in settings["mosquito_species"]:
            # print(f"🌍 Matched Environment: {env} for {species}")
            return env  

    # print("⚠️ No matching environment found, defaulting to 'urban'")
    return "urban"


def get_noise_files(noise_dir, environment_type):
    """
    Retrieve available noise files based on environment type.
    """
    noise_subdir = config.ENVIRONMENTS[environment_type]["noise_subdir"]
    noise_path = os.path.join(noise_dir, noise_subdir)

    if not os.path.exists(noise_path):
        raise ValueError(f"Missing noise directory: {noise_path}")
    
    noise_files = [f for f in os.listdir(noise_path) if f.endswith(".wav")]
    
    if not noise_files:
        raise ValueError(f"No noise files found in {noise_path}")
    
    return noise_files, noise_path



def apply_smooth_volume(audio, sr, fade_in_time=1.0, fade_out_time=1.0, method="sigmoid"):
    fade_in_samples = min(max(int(fade_in_time * sr), 1), len(audio) // 2)
    fade_out_samples = min(max(int(fade_out_time * sr), 1), len(audio) // 2)

    if len(audio) < 2 * max(fade_in_samples, fade_out_samples):
        print(f"⚠️ Warning: Audio is too short for requested fade-in/out times. Skipping fade.")
        return audio  

    fade_in_curve = np.ones(fade_in_samples)
    fade_out_curve = np.ones(fade_out_samples)

    if method == "linear":
        fade_in_curve = np.linspace(0, 1, fade_in_samples)
        fade_out_curve = np.linspace(1, 0, fade_out_samples)

    elif method == "exponential":
        fade_in_curve = (np.exp(np.linspace(0, 4, fade_in_samples)) - 1) / (np.exp(4) - 1)
        fade_out_curve = np.flip(fade_in_curve)

    elif method == "sigmoid":
        x = np.linspace(-6, 6, fade_in_samples)
        fade_in_curve = 1 / (1 + np.exp(-x))
        fade_out_curve = np.flip(fade_in_curve)

    # Apply fade-in and fade-out
    dynamic_audio = audio.copy()
    dynamic_audio[:fade_in_samples] *= fade_in_curve
    dynamic_audio[-fade_out_samples:] *= fade_out_curve

    return dynamic_audio



# def prep_env(noise_dir, mosquito_dirs):

#     selected_file, species = get_mosquito_species(mosquito_dirs)

#     # 🔹 กำหนด environment จาก species ที่สุ่มมา
#     env_type = get_environment_type(species)
    
#     noise_files, noise_path = get_noise_files(noise_dir, env_type)
#     noise_file = random.choice(noise_files)
#     noise_filepath = os.path.join(noise_path, noise_file)
    
#     background_noise, _ = librosa.load(noise_filepath, sr=config.SAMPLING_RATE, mono=True)    
    
#     total_samples = int(config.AUDIO_DURATION * config.SAMPLING_RATE)
#     if len(background_noise) > total_samples:
#         start_idx = random.randint(0, len(background_noise) - total_samples)
#         background_noise = background_noise[start_idx:start_idx + total_samples]
#     else:
#         background_noise = np.pad(background_noise, (0, total_samples - len(background_noise)), mode='wrap')

#     subenv = noise_file.split('_')[2]  # Extract sub-environment

#         # ✅ ปรับระดับเสียงรบกวนโดยใช้ noise_level
#     noise_level = config.ENVIRONMENTS[env_type]["noise_level"]
#     background_noise *= noise_level  
#     # ✅ ลดเสียงพื้นหลังเพิ่ม (Exponential Scaling)
#     background_noise = np.sign(background_noise) * np.abs(background_noise) ** 0.8
#     background_noise /= np.max(np.abs(background_noise))  # Normalize
    
    
#     return background_noise, env_type, noise_file, subenv

# def generate_random_pos(duration, min_events=2, min_interval=0.2, mean_interval=1.0, 
#                         noise_prob=0.5, mosquito_clump_prob=0.1):
#     intervals = []
#     current_time = 0.0

#     while current_time < duration:
#         first_event = "noise" if random.random() < noise_prob else "mosquito"

#         interval_length = max(min_interval, random.expovariate(1.0 / mean_interval))
#         if current_time + interval_length > duration:
#             break

#         end_time = current_time + interval_length

#         if first_event == "noise":
#             # ✅ Noise ก่อน แล้ว Mosquito ตาม
#             intervals.append(("Noise", current_time, end_time))
#             current_time = end_time  # Noise จบแล้วไป Mosquito ต่อ
            
#             # ✅ กำหนด Mosquito หลังจาก Noise
#             mos_start = current_time + random.uniform(min_interval, mean_interval)
#             if mos_start < duration:
#                 mos_end = mos_start + max(min_interval, random.expovariate(1.0 / mean_interval))
#                 if mos_end > duration:
#                     mos_end = duration
#                 intervals.append(("Mosquito", mos_start, mos_end))
#                 current_time = mos_end  # ไป Noise หรือ Mosquito ต่อ

#                 # ✅ มีโอกาสให้ Mosquito Events ติดกัน
#                 while random.random() < mosquito_clump_prob and current_time < duration:
#                     next_start = current_time + random.uniform(min_interval, mean_interval / 2)
#                     if next_start > duration:
#                         break
#                     next_end = next_start + max(min_interval, random.expovariate(1.0 / mean_interval))
#                     if next_end > duration:
#                         next_end = duration
#                     intervals.append(("Mosquito", next_start, next_end))
#                     current_time = next_end  # ไป Mosquito ต่อ

#         else:
#             # ✅ Mosquito ก่อน แล้ว Noise ตาม
#             intervals.append(("Mosquito", current_time, end_time))
#             current_time = end_time  # Mosquito จบแล้วไป Noise ต่อ

#             # ✅ Noise หลังจาก Mosquito
#             noise_start = current_time + random.uniform(min_interval, mean_interval)
#             if noise_start < duration:
#                 noise_end = noise_start + max(min_interval, random.expovariate(1.0 / mean_interval))
#                 if noise_end > duration:
#                     noise_end = duration
#                 intervals.append(("Noise", noise_start, noise_end))
#                 current_time = noise_end  # ไป Mosquito ต่อ

#     # ✅ ตรวจสอบว่ามีอย่างน้อย `min_events` หรือไม่
#     mosquito_events = [event for event in intervals if event[0] == "Mosquito"]
#     while len(mosquito_events) < min_events:
#         last_end_time = intervals[-1][2] if intervals else 0.0
#         new_start = last_end_time + random.uniform(min_interval, mean_interval)
#         if new_start + min_interval > duration:
#             break  # ✅ หยุดเพิ่มถ้าเกิน 10 วินาที

#         new_end = new_start + random.uniform(min_interval, mean_interval)
#         intervals.append(("Mosquito", new_start, min(duration, new_end)))  # ✅ ห้ามเกิน 10 วิ
#         mosquito_events.append(("Mosquito", new_start, new_end))

#     return sorted(intervals, key=lambda x: x[1])  # ✅ เรียงลำดับตามเวลาเริ่มต้น


def prep_env(noise_dir, mosquito_dirs, env_target):
    """
    ปรับ prep_env เพื่อบังคับให้เลือก environment ตาม env_target
    """
    # กรอง mosquito files ที่ตรงกับ env_target โดยใช้ config.ENVIRONMENTS
    matching_files = []
    for mosquito_dir in mosquito_dirs:
        if not os.path.exists(mosquito_dir):
            continue
        for f in os.listdir(mosquito_dir):
            if f.endswith(".wav") and not f.startswith("An.Minimus"):
                species = f.split("_")[0]
                # ตรวจสอบว่า species นี้อยู่ในกลุ่มของ env_target หรือไม่
                if species in config.ENVIRONMENTS[env_target]["mosquito_species"]:
                    matching_files.append(os.path.join(mosquito_dir, f))
    if not matching_files:
        raise ValueError(f"No mosquito files found for environment: {env_target}")
    selected_file = random.choice(matching_files)
    species = os.path.basename(selected_file).split("_")[0]
    env_type = env_target  # บังคับใช้ env_target

    # ดึง noise files สำหรับ env_target
    noise_files, noise_path = get_noise_files(noise_dir, env_target)
    noise_file = random.choice(noise_files)
    noise_filepath = os.path.join(noise_path, noise_file)
    
    background_noise, _ = librosa.load(noise_filepath, sr=config.SAMPLING_RATE, mono=True)
    total_samples = int(config.AUDIO_DURATION * config.SAMPLING_RATE)
    if len(background_noise) > total_samples:
        start_idx = random.randint(0, len(background_noise) - total_samples)
        background_noise = background_noise[start_idx:start_idx + total_samples]
    else:
        background_noise = np.pad(background_noise, (0, total_samples - len(background_noise)), mode='wrap')

    subenv = noise_file.split('_')[2]  # Extract sub-environment
    noise_level = config.ENVIRONMENTS[env_target]["noise_level"]
    background_noise *= noise_level  
    background_noise = np.sign(background_noise) * np.abs(background_noise) ** 0.8
    background_noise /= np.max(np.abs(background_noise))
    
    return background_noise, env_type, noise_file, subenv


def generate_random_pos(duration, min_interval=0.1, mean_interval=2.0, noise_prob=0.7, mosquito_prob=0.5, mosquito_clump_prob=0.2):
    """
    Generate random event positions with significant noise-only periods covering the full duration.
    
    Args:
        duration (float): Total duration of the audio in seconds.
        min_interval (float): Minimum duration of an event.
        mean_interval (float): Mean interval between event starts (increased for sparsity).
        noise_prob (float): Probability of an event being noise (higher for more noise).
        mosquito_prob (float): Probability of including mosquito events.
        mosquito_clump_prob (float): Probability of mosquito events clumping together.
    
    Returns:
        List of tuples: (event_type, start_time, end_time) covering 0 to duration, sorted by start time.
    """
    intervals = []
    current_time = 0.0

    # Decide if this simulation includes mosquito events (50% chance)
    has_mosquitoes = random.random() < mosquito_prob

    # Limit total events for sparsity (1–4 events)
    total_events = random.randint(1, 4)
    event_count = 0

    while event_count < total_events and current_time < duration:
        # Random start time for the next event
        gap = np.random.exponential(1.0 / mean_interval)
        start_time = min(current_time + gap, duration)
        if start_time >= duration:
            break

        # Define event duration (short, max 0.5s)
        event_duration = random.uniform(min_interval, 0.5)
        end_time = min(start_time + event_duration, duration)

        # Fill gap before this event with "Noise" if it exists
        if start_time > current_time:
            intervals.append(("Noise", current_time, start_time))

        # Assign event type
        event_type = "noise" if not has_mosquitoes or random.random() < noise_prob else "mosquito"
        intervals.append((event_type, start_time, end_time))
        current_time = end_time
        event_count += 1

        # Add clumping for mosquito events
        if has_mosquitoes and event_type == "mosquito" and random.random() < mosquito_clump_prob and event_count < total_events:
            clump_duration = random.uniform(min_interval, 0.3)
            clump_end = min(current_time + clump_duration, duration)
            if clump_end > current_time:
                intervals.append(("mosquito", current_time, clump_end))
                current_time = clump_end
                event_count += 1

    # Fill remaining time with "Noise" if needed
    if current_time < duration:
        intervals.append(("Noise", current_time, duration))

    # Ensure at least one mosquito event if none exist (low chance)
    mosquito_events = [e for e in intervals if e[0] == "mosquito"]
    if has_mosquitoes and not mosquito_events and random.random() < 0.2:
        start = random.uniform(0, duration - min_interval)
        end = min(start + random.uniform(min_interval, 0.5), duration)
        # Insert mosquito event and adjust surrounding noise periods
        new_intervals = []
        for event_type, s, e in intervals:
            if s < start < e:
                new_intervals.append(("Noise", s, start))
                new_intervals.append(("mosquito", start, end))
                if end < e:
                    new_intervals.append(("Noise", end, e))
            elif end <= s or start >= e:
                new_intervals.append((event_type, s, e))
        intervals = new_intervals
    else:
        # If no adjustment needed, just sort
        intervals = sorted(intervals, key=lambda x: x[1])

    return intervals


def prepare_mos(environment_type, mosquito_dirs, background_noise, event_intervals):
    """
    Generate mosquito audio signals with randomized events and metadata.
    """
    env_settings = config.ENVIRONMENTS[environment_type]
    allowed_species = env_settings["mosquito_species"]
    snr_range = env_settings["snr_range"]

    random.seed(None)  
    np.random.seed(None)

    mosquito_files = []
    for mosquito_dir in mosquito_dirs:
        for f in os.listdir(mosquito_dir):
            if f.endswith(".wav") and f.split("_")[0] in allowed_species:
                mosquito_files.append(os.path.join(mosquito_dir, f))

    total_samples = int(config.AUDIO_DURATION * config.SAMPLING_RATE)
    random.shuffle(mosquito_files)

    mosquito_audio = np.zeros(total_samples)
    metadata = {"audio_labels": [], "event_intervals": []}
    snr_values = np.random.uniform(snr_range[0], snr_range[1], size=len(event_intervals))

    previous_end = 0  # Track last event end time

    for (event_type, start, end), snr in zip(event_intervals, snr_values):
        start_sample, end_sample = int(start * config.SAMPLING_RATE), int(end * config.SAMPLING_RATE)

        # ✅ กรณีเป็น Noise Event → แค่บันทึก metadata (ไม่ต้องเพิ่มเสียง)
        if event_type == "Noise":
            metadata["audio_labels"].append({
                "event_type": "Noise",
                "start_time": start,
                "end_time": end,
                "species": None,
                "sex": None,
                "snr": None,
                "environment": environment_type
            })
            previous_end = end  # ✅ อัปเดตตำแหน่งล่าสุด
            continue  

        # ✅ ถ้าเป็น Mosquito Event → โหลดไฟล์และเพิ่มเสียง
        mosquito_file = random.choice(mosquito_files)
        filename = os.path.basename(mosquito_file)
        
        parts = filename.split("_")
        species_name = parts[0] if len(parts) > 0 else "Unknown"
        sex = parts[1][1] if len(parts) > 1 else "Unknown"

        mosquito_sound, _ = librosa.load(mosquito_file, sr=config.SAMPLING_RATE, mono=True)

        mosquito_duration = end_sample - start_sample
        mosquito_sound = np.pad(mosquito_sound[:mosquito_duration], (0, max(0, mosquito_duration - len(mosquito_sound))), 'constant')

        
        mosquito_sound = apply_smooth_volume(mosquito_sound, config.SAMPLING_RATE, fade_in_time=0.7, fade_out_time=0.7, method="exponential")

        
        
        # ✅ คำนวณ Scaling Factor ให้ SNR อยู่ในช่วงที่กำหนด
        noise_power = np.mean(background_noise[start_sample:end_sample] ** 2)
        mosquito_power = np.mean(mosquito_sound ** 2)
        snr_linear = 10 ** (snr / 10)
        scaling_factor = np.sqrt(noise_power * snr_linear / (mosquito_power + 1e-9))

        scaling_factor = np.sqrt(noise_power * snr_linear / (mosquito_power + 1e-9))
        mosquito_sound *= scaling_factor * np.random.uniform(0.9, 1.1)


        mosquito_audio[start_sample:end_sample] += mosquito_sound


        # ✅ บันทึก Mosquito Event
        metadata["audio_labels"].append({
            "event_type": "Mosquito",
            "start_time": start,
            "end_time": end,
            "species": species_name,
            "sex": sex,
            "snr": round(snr, 2),
            "environment": environment_type
        })

        previous_end = end  # ✅ อัปเดตตำแหน่งล่าสุด

    return mosquito_audio, metadata





# def process_simulation(mosquito_dirs, noise_dir, output_dir, num_simulations,env_target):
#     """
#     Generate simulated mosquito sound data.
#     """
#     os.makedirs(output_dir, exist_ok=True)

#     metadata_list = []
#     random.seed(None)
#     np.random.seed(None)

#     if isinstance(mosquito_dirs, set):
#         mosquito_dirs = list(mosquito_dirs)

#     for i in range(num_simulations):
#         background_noise, env_type, noise_file, subenv = prep_env(noise_dir, mosquito_dirs,env_target)
#         mosquito_intervals = generate_random_pos(config.AUDIO_DURATION)
#         mosquito_audio, metadata = prepare_mos(env_type, mosquito_dirs, background_noise, mosquito_intervals)

#         # ✅ รวมเสียงยุง + Background Noise
#         simulated_audio = background_noise + mosquito_audio

#         # # ✅ ใช้ Crossfade เพื่อลดเสียงกระโดด
#         # simulated_audio = apply_crossfade(simulated_audio, fade_samples=500)

#         # ✅ ปรับความดังให้คงที่ (Normalize)
#         simulated_audio = normalize(simulated_audio)

#         # ✅ บันทึกไฟล์เสียงที่ถูก Generate
#         audio_filename = f"{env_type}_simulated_{i+1}.wav"
#         audio_filepath = os.path.join(output_dir, audio_filename)
#         sf.write(audio_filepath, simulated_audio, config.SAMPLING_RATE)
        
        
#         if mosquito_intervals:
#             last_mosquito = mosquito_intervals[-1]
#       # ✅ ตรวจสอบว่า Mosquito ตัวสุดท้ายไปจนถึง AUDIO_DURATION หรือไม่
#             if last_mosquito[2] < config.AUDIO_DURATION:
#                 last_noise_exists = any(
#                     entry["event_type"] == "Noise" and entry["start_time"] == last_mosquito[2] 
#                     for entry in metadata["audio_labels"]
#                 )

#                 if not last_noise_exists:
#                     metadata["audio_labels"].append({
#                         "event_type": "Noise",
#                         "start_time": last_mosquito[2],  # ✅ Noise เริ่มที่เวลาสิ้นสุดของ Mosquito สุดท้าย
#                         "end_time": config.AUDIO_DURATION,
#                         "species": None,
#                         "sex": None,
#                         "snr": None,
#                         "environment": env_type
#                     })

#         for entry in metadata["audio_labels"]:
#             entry["file_path"]= audio_filepath
#             entry["file_name"] = audio_filename
#             entry["duration"] = librosa.get_duration(y=simulated_audio, sr=config.SAMPLING_RATE)
#             entry["noise_file"] = noise_file
#             entry["subenv"] = subenv
#             metadata_list.append(entry)

#     metadata_df = pd.DataFrame(metadata_list)

#     # ✅ Save Metadata
#     # metadata_csv_path = os.path.join(config.METADATA_DIR, "simulation_metadata.csv")
#     # metadata_df.to_csv(metadata_csv_path, index=False)

#     # print("\nMetadata Preview:")
#     # print(metadata_df.head(20))  

#     return simulated_audio, metadata_df
      
def process_simulation(mosquito_dirs, noise_dir, output_dir, num_simulations, env_target):
    """
    Generate simulated mosquito sound data.
    """
    os.makedirs(output_dir, exist_ok=True)

    metadata_list = []
    random.seed(None)
    np.random.seed(None)

    if isinstance(mosquito_dirs, set):
        mosquito_dirs = list(mosquito_dirs)

    for i in range(num_simulations):
        background_noise, env_type, noise_file, subenv = prep_env(noise_dir, mosquito_dirs, env_target)
        mosquito_intervals = generate_random_pos(config.AUDIO_DURATION)
        mosquito_audio, metadata = prepare_mos(env_type, mosquito_dirs, background_noise, mosquito_intervals)

        # Combine audio
        simulated_audio = background_noise + mosquito_audio
        simulated_audio = normalize(simulated_audio)

        # Save audio file
        audio_filename = f"{env_type}_simulated_{i+1}.wav"
        audio_filepath = os.path.join(output_dir, audio_filename)
        sf.write(audio_filepath, simulated_audio, config.SAMPLING_RATE)

        # Add file metadata to each entry
        for entry in metadata["audio_labels"]:
            entry["file_path"] = audio_filepath
            entry["file_name"] = audio_filename
            entry["duration"] = librosa.get_duration(y=simulated_audio, sr=config.SAMPLING_RATE)
            entry["noise_file"] = noise_file
            entry["subenv"] = subenv
            metadata_list.append(entry)

    metadata_df = pd.DataFrame(metadata_list)
    return simulated_audio, metadata_df


def main():
    # Define directories
    mosquito_dirs = list({config.INDOOR_DIR, config.OUTDOOR_DIR})
    noise_dir = config.NOISE_VALTEST_DIR
    output_dir = config.SIMULATED_DIR
    env_target = "urban"  # Change this if you want to simulate another environment

    # Run the simulation but do not save
    print("\n🚀 Running Simulation...")
    
    for _ in range(5):  # Run 5 simulations
        simulated_audio, metadata_df = process_simulation(
            mosquito_dirs, noise_dir, output_dir, num_simulations=1, env_target=env_target
        )

        # Display metadata
        print("\n📊 Metadata Preview:")
        print(metadata_df)
        
        # Play audio directly
        print("\n🔊 Playing Generated Audio...")
        display(ipd.Audio(simulated_audio, rate=config.SAMPLING_RATE))

# Run the main function
main()



🚀 Running Simulation...



📊 Metadata Preview:
  event_type  start_time   end_time     species   sex    snr environment  \
0   Mosquito    0.000000   0.324709  Ae.Aegypti     F   8.74       urban   
1   Mosquito    0.655878   0.792502  Ae.Aegypti     M   6.25       urban   
2   Mosquito    1.435046   1.664105  Ae.Aegypti     F   9.93       urban   
3   Mosquito    2.402500   2.637694  Ae.Aegypti     M  13.26       urban   
4   Mosquito    2.637694   2.806885     Cx.Quin     F  12.99       urban   
5      Noise    2.806885  10.000000        None  None    NaN       urban   

                                           file_path              file_name  \
0  /media/mosdet_nas/mosdet/Mosqui-DST/data/simul...  urban_simulated_1.wav   
1  /media/mosdet_nas/mosdet/Mosqui-DST/data/simul...  urban_simulated_1.wav   
2  /media/mosdet_nas/mosdet/Mosqui-DST/data/simul...  urban_simulated_1.wav   
3  /media/mosdet_nas/mosdet/Mosqui-DST/data/simul...  urban_simulated_1.wav   
4  /media/mosdet_nas/mosdet/Mosqui-DST/data/simul..


📊 Metadata Preview:
   event_type  start_time   end_time     species   sex    snr environment  \
0    Mosquito    0.000000   0.107932     Cx.Quin     M   9.91       urban   
1    Mosquito    1.058814   1.486821     Cx.Quin     F  11.37       urban   
2    Mosquito    2.312458   2.631465     Cx.Quin     M   7.37       urban   
3    Mosquito    2.928744   3.339303  Ae.Aegypti     M   8.26       urban   
4    Mosquito    4.075976   4.204862  Ae.Aegypti     M   6.94       urban   
5    Mosquito    5.104249   5.525921     Cx.Quin     F   9.22       urban   
6    Mosquito    6.124365   6.418515  Ae.Aegypti     F   7.73       urban   
7    Mosquito    7.334315   7.560207  Ae.Aegypti     M  13.78       urban   
8    Mosquito    8.499098   8.634243  Ae.Aegypti     M   6.28       urban   
9    Mosquito    9.080289   9.296500  Ae.Aegypti     F  12.74       urban   
10      Noise    9.296500  10.000000        None  None    NaN       urban   

                                            file_path 


📊 Metadata Preview:
   event_type  start_time   end_time     species   sex    snr environment  \
0    Mosquito    0.000000   0.084670     Cx.Quin     M   8.44       urban   
1    Mosquito    0.757167   1.007062  Ae.Aegypti     M   9.64       urban   
2    Mosquito    1.399238   1.877484  Ae.Aegypti     M   9.29       urban   
3    Mosquito    2.182812   2.434954  Ae.Aegypti     M   8.91       urban   
4    Mosquito    3.266106   3.440387     Cx.Quin     F   8.55       urban   
5    Mosquito    3.931824   4.088606  Ae.Aegypti     M   5.68       urban   
6    Mosquito    4.652772   5.065903  Ae.Aegypti     M  10.09       urban   
7    Mosquito    5.597816   5.779446  Ae.Aegypti     F  11.32       urban   
8    Mosquito    6.001972   6.353219     Cx.Quin     F  12.83       urban   
9    Mosquito    6.975721   7.211897  Ae.Aegypti     M  13.39       urban   
10   Mosquito    8.121543   8.252726  Ae.Aegypti     F   9.81       urban   
11      Noise    8.252726  10.000000        None  None 


📊 Metadata Preview:
   event_type  start_time   end_time     species   sex    snr environment  \
0    Mosquito    0.000000   0.409801  Ae.Aegypti     M  11.55       urban   
1    Mosquito    1.190984   1.292975     Cx.Quin     F   7.67       urban   
2    Mosquito    2.211512   2.495033  Ae.Aegypti     F  14.52       urban   
3    Mosquito    2.728253   3.182738     Cx.Quin     F   7.77       urban   
4    Mosquito    3.507746   3.607837     Cx.Quin     M   8.94       urban   
5    Mosquito    3.921693   4.387225  Ae.Aegypti     F   7.83       urban   
6    Mosquito    4.856960   5.021927  Ae.Aegypti     F   6.69       urban   
7    Mosquito    5.309057   5.696250     Cx.Quin     M   7.66       urban   
8    Mosquito    6.505464   6.614589  Ae.Aegypti     M   5.73       urban   
9    Mosquito    7.029382   7.473792  Ae.Aegypti     F   7.73       urban   
10   Mosquito    7.965399   8.340839     Cx.Quin     F  14.64       urban   
11   Mosquito    8.928320   9.012302     Cx.Quin     F 


📊 Metadata Preview:
   event_type  start_time   end_time     species   sex    snr environment  \
0    Mosquito    0.000000   0.384899  Ae.Aegypti     M   8.47       urban   
1    Mosquito    0.644154   1.086201  Ae.Aegypti     F   5.45       urban   
2    Mosquito    1.466040   1.766153     Cx.Quin     M  13.65       urban   
3    Mosquito    2.695950   3.137451  Ae.Aegypti     M   9.17       urban   
4    Mosquito    3.902758   4.223809     Cx.Quin     F  13.95       urban   
5    Mosquito    4.906736   5.127240  Ae.Aegypti     M  10.23       urban   
6    Mosquito    5.818910   6.163997  Ae.Aegypti     M  13.78       urban   
7    Mosquito    6.651678   6.774941  Ae.Aegypti     M   7.67       urban   
8    Mosquito    6.978494   7.027096  Ae.Aegypti     M   9.18       urban   
9    Mosquito    7.776958   7.883061     Cx.Quin     M   9.31       urban   
10   Mosquito    8.200799   8.690757  Ae.Aegypti     M  11.95       urban   
11   Mosquito    9.644938   9.742282     Cx.Quin     M 